# 이진 분류: 비선형 회귀

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x1 = df.iloc[:, 0].values
    self.x2 = df.iloc[:, 1].values
    self.x3 = df.iloc[:, 2].values
    self.y = df.iloc[:, 3].values
    self.length = len(df)

  def __getitem__(self, index):
    x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
    y = torch.FloatTensor([int(self.y[index])])
    return x, y

  def __len__(self):
    return self.length

In [ ]:
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer = nn.Sequential(  # 여러 계층을 하나로 묶기
        nn.Linear(3, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    x = self.layer(x)
    return x

In [ ]:
dataset = CustomDataset("/content/drive/MyDrive/Colab Notebooks/아이펠/AIFFEL_practice/pytorch/base/data/binary.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)  # 이진 교차 엔트로피
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [ ]:
for epoch in range(10000):
  cost = 0.0

  for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost += loss

  cost = cost / len(train_dataloader)

  if (epoch + 1) % 1000 == 0:
    print(f"Epoch: {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch: 1000, Model : [Parameter containing:
tensor([[0.0053, 0.0001, 0.0019]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.0186], device='cuda:0', requires_grad=True)], Cost : 0.668
Epoch: 2000, Model : [Parameter containing:
tensor([[0.0060, 0.0013, 0.0028]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.1071], device='cuda:0', requires_grad=True)], Cost : 0.653
Epoch: 3000, Model : [Parameter containing:
tensor([[0.0069, 0.0022, 0.0036]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.2298], device='cuda:0', requires_grad=True)], Cost : 0.642
Epoch: 4000, Model : [Parameter containing:
tensor([[0.0072, 0.0029, 0.0042]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.3497], device='cuda:0', requires_grad=True)], Cost : 0.630
Epoch: 5000, Model : [Parameter containing:
tensor([[0.0065, 0.0024, 0.0035]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.4668], device='

In [ ]:
with torch.no_grad():
  model.eval()
  for x, y, in validation_dataloader:
    x = x.to(device)
    y = y.to(device)

    outputs = model(x)

    print(outputs)
    print(outputs >= torch.FloatTensor([0.5]).to(device))
    print("----------------")

tensor([[0.5050],
        [0.6788],
        [0.4741],
        [0.6005]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [ True]], device='cuda:0')
----------------
tensor([[0.6686],
        [0.5917],
        [0.5539],
        [0.4430]], device='cuda:0')
tensor([[ True],
        [ True],
        [ True],
        [False]], device='cuda:0')
----------------
tensor([[0.5371],
        [0.4769],
        [0.7488],
        [0.6208]], device='cuda:0')
tensor([[ True],
        [False],
        [ True],
        [ True]], device='cuda:0')
----------------
tensor([[0.6321],
        [0.6183],
        [0.4575],
        [0.5052]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [ True]], device='cuda:0')
----------------
tensor([[0.6359],
        [0.7214],
        [0.5668],
        [0.5945]], device='cuda:0')
tensor([[True],
        [True],
        [True],
        [True]], device='cuda:0')
----------------
tensor([[0.5335],
        [0.6565],
   